# Paper plots
Scripts use to create plots in the paper


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors
import pandas as pd
import chemistry as chem
import func
from plotting import HarryPlotter as plotter
import sys
from data import DataHandler
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
calc=func.Calculator

plt.rcParams['xtick.minor.visible'], plt.rcParams['xtick.top'] = True, True
plt.rcParams['ytick.minor.visible'], plt.rcParams['ytick.right'] = True, True
plt.rcParams['xtick.direction'], plt.rcParams['ytick.direction'] = 'in', 'in'
plt.rcParams['mathtext.fontset'] = "stix"
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams["font.size"] = 16

fig_width = 5

linewidth=2.5
filetype='pdf'

if filetype =='png':
    folder = 'presentation/'
else:
    folder=''
    
solar_abun = pd.read_csv("solar_abun.csv")
old_solar_abun = pd.read_csv('old_solar_abun.csv')
full_SAPP_data = pd.read_csv('SAPP_total_data_Jesper_final_snr_cut_50.csv')
species_data = pd.read_csv("species_data.csv",delimiter=";")


min_mass = 0.05
M_star_lim = 1.4
giant_lim = 10
SE_uplim = 10
SE_lowlim = 2
gas_lim = 0.15
wfrac_lim = 0.1
MS2ME = 1.9891*10**30/(5.9722*10**24)
Mps2AUpYr = 2.108*10**-4

theta = np.array([[0.01,0.01,0.01,1*MS2ME,3/7,15/14,0.005,0.0001,2]]).T

abun = chem.Abundance(solar_abun,carbon_grain=True,free_iron=True)

track = calc(theta,abun=solar_abun)


In [ ]:
#%% Plot disc lifetimes and sizes

M_star_vals = np.linspace(0.1,1.4,100)

t_gas = np.zeros(100)
R1 = np.zeros(100)

for i,M_star in enumerate(M_star_vals):
    
    track.M_star = np.array([M_star*MS2ME])
    track.M_gas = 0.1*(M_star)**1.4*MS2ME
    t_gas[i] = track.get_disc_lifetime()
    R1[i] = track.get_R1()

fig,ax1 = plt.subplots(figsize=(fig_width,fig_width-1))

ax1.plot(M_star_vals,t_gas/10**6,color='black',label = 'Disc Lifetime')
ax1.set_xlabel('$M_* [M_\odot]$')
ax1.set_ylabel('Disc lifetime [Myr]')
#ax1.set_xscale('log')

ax2=ax1.twinx()

ax2.plot(0,0, color='black',label = 'Disc lifetime')
ax2.plot(M_star_vals,R1,color='black',linestyle = '--',label = 'Disc size')
ax2.set_ylabel('Disc size [AU]',labelpad=0)
ax2.set_yscale('log')
#ax2.set_xscale('log')

plt.legend(loc=2,fontsize=14)
plt.tight_layout()

plt.savefig('Figures/{}disc_lifetimes_sizes.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% Plot metallicity

abun_freeiron = chem.Abundance(solar_abun,free_iron = True)

R1 = track.get_R1()
t_gas = track.get_disc_lifetime()

r_vals = np.logspace(-1.2,np.log10(200),1000)

#times = np.linspace(0, t_gas,5)[:,0]


T = track.T(r_vals,2*10**6)

T_lines = [20,70,90,180,480]

labels=[r'CO ice line',
        r'CO$_2$ ice line',
        r'NH$_3$ ice line',
        r'Water ice line',
        r'Fayalite formation']

line_idx = np.array([func.find_nearest(T[:,0],i) for i in T_lines])
r_lines = r_vals[line_idx]

solid_frac = abun.get_solid_frac(T.reshape(1000,1))
solid_frac_freeiron = abun_freeiron.get_solid_frac(T.reshape(1000,1))

plt.figure(figsize=(fig_width,fig_width))
plt.plot(r_vals,solid_frac,
         color = 'red',label='Iron oxidation')
plt.plot(r_vals,solid_frac_freeiron,
         color='blue',linestyle='-',label = 'No Iron oxidation')
y_min,y_max = plt.gca().get_ylim()

for i_r,r_val in enumerate(r_lines):
   
    plt.vlines(r_val,y_min,y_max,
               color='grey',linestyle='dashed',alpha=0.8)
    
    plt.text(r_val,y_max,labels[i_r],rotation=90,
             color='grey',alpha=0.8,fontsize=14,
             verticalalignment='top',horizontalalignment='right')

plt.vlines(track.R1[0,0],y_min,y_max,
           color='grey',linestyle='dashed',alpha=0.8,)

plt.text(track.R1[0,0],y_max,'Disc size',rotation=90,fontsize=14,
         color='grey',alpha=0.8,verticalalignment='top',horizontalalignment='right')

plt.legend(loc=4,fontsize=14)
plt.xlabel('Distance to star [AU]')
plt.ylabel('Dust-to-gas ratio')
plt.xscale('log')
plt.tight_layout()
plt.savefig('Figures/{}metallicity_r.{}'.format(folder,filetype))

In [ ]:
#%% Plot pebble flux over time - calculations

r = 10
M_star=1

tmin = 3*10**5
tmin =0
#M_gas = track.M_gas(0)
M_gas = 0.1*M_star**1.4
dt = 5000
t_end = np.max(track.get_disc_lifetime())
abun=chem.Abundance(solar_abun)

times = np.arange(tmin,t_end,dt).astype(float)

M_gdot = track.M_gdot(times)

M_pdot_vals = np.zeros((3,len(times)))
St_vals = np.zeros((3,len(times)))
ratio_vals = np.zeros((3,len(times)))

#r_list = [10]
r_list = [1,10,50]

for i_r,r in enumerate(r_list):
    
    print('{} AU'.format(r))
    M_pdot,St = track.M_pdot_joanna(r,times,store_St=True)
    
    M_pdot_vals[i_r] = M_pdot[:,0]
    St_vals[i_r] = St[:,0]
    
    ratio = M_pdot/M_gdot
    
    ratio_vals[i_r] = ratio[:,0]

#%% Plot pebble flux over time - plotting


colors = ['black','blue','red','purple']

fig,axes=plt.subplots(2,1,figsize=(fig_width,fig_width*1.5),sharex=True)
ax1=axes[0]

ax2=ax1.twinx()

for i_r,r in enumerate(r_list):
    
    ax1.plot(times,M_pdot_vals[i_r],color=colors[i_r])
    ax2.plot(times,ratio_vals[i_r],color = colors[i_r],linestyle='--')
    
    axes[1].plot(times,St_vals[i_r],color=colors[i_r],label = '{} AU'.format(r))

axes[0].text(0.05,0.1,'(a)',transform=axes[0].transAxes)
axes[1].text(0.05,0.1,'(b)',transform=axes[1].transAxes)

ax1.set_xscale('log')
ax1.set_yscale('log')
ax2.set_yscale('log')
axes[1].set_yscale('log')

ax1.plot([],[],color='grey',label=r'$\dot{M}_{\rm p}/\dot{M}_{\rm g}$',linestyle='--')
ax1.plot([],[],color = 'grey',label=r'$\dot{M}_{\rm p}$',linestyle='-')
ax1.set_xlim(10**3,ax1.get_xlim()[1])
ax1.legend(loc='upper right')
axes[1].legend(loc='lower right')

ax1.set_ylabel(r'Pebble flux [$M_\oplus{\rm yr}^{-1}$]')
ax2.set_ylabel(r'Flux ratio')

axes[1].set_ylabel('Stokes number')
axes[1].set_xlabel('Time [Yr]')
plt.tight_layout()
plt.savefig("Figures/{}pebble_flux_st.{}".format(folder,filetype))  
plt.close()

In [ ]:
#%% Plot fraction of solids

N_points = 3000
r_vals = np.logspace(-2,2,N_points)
T = track.T(r_vals,0).reshape(N_points,1)
model_label=['Excluding Iron oxidation','Including Iron oxidation']

#colors = ["red","blue","y","brown","black","yellow",
#          "magenta","royalblue","teal","forestgreen","pink","orange"]

fig,axes = plt.subplots(2,1,figsize = (fig_width,fig_width*1.5),sharex=True)

for i_plot, model in enumerate([True,False]):    

    ax=axes[i_plot]    

    abun = chem.Abundance(solar_abun,free_iron=model)
    vfrac = abun.get_vol_frac(T)
    spec_names = abun.spec_names
    
    vfrac_vals = np.array(list(vfrac.values()))[:,:,0].T
    
    bad_list = ["He","H2",'N2','C-O','Mg','Si-O','H2-S','Si','Fe3-O4','C-H4']
    
    bad_filt = [i in bad_list for i in spec_names]
    
    vfrac_vals = np.delete(vfrac_vals,np.where(bad_filt)[0],axis=1)
    
    plot_names = np.delete(spec_names,np.where(bad_filt)[0])
    
    evap_temps = np.delete(abun.temps,np.where(bad_filt)[0])
    
    mu = np.delete(abun.mu,np.where(bad_filt)[0])
    
    gas = (evap_temps != 0) & (T>=evap_temps)
    
    vfrac_solid = np.copy(vfrac_vals)
    
    vfrac_solid[gas] = 0
    
    vfrac_dist = vfrac_solid*mu/np.sum(vfrac_vals*mu,axis=1,keepdims=True)
    
    vfrac_dist[-1] = 0
    
    vfrac_plot = np.cumsum(vfrac_dist,axis=1)
    
    prev = 0
    
    colors = iter(plt.cm.tab20b(np.linspace(0, 1,len(plot_names))))
    
    plot_labels = func.rename_species(plot_names)
    
    if i_plot==0:
        height=0.65
    else:
        height=0.9
    ax.text(0.03,height,model_label[i_plot],transform=ax.transAxes,fontsize=11)
    
    for i, spec in enumerate(plot_names):
    
        y_vals = vfrac_plot[:,i]
        
        temps_idx = np.where(spec_names == spec)[0][0]
        c= next(colors)
        ax.fill_between(r_vals,y_vals,prev,label=plot_labels[i],color=c,edgecolor='black')
        #plt.plot(T,vfrac_plot[:,i],label=spec)
        prev = y_vals

axes[0].set_xscale('log')
axes[0].legend(loc=2,ncol=3,frameon=False,fontsize = 9)
axes[1].set_xlabel("Distance to star [AU]")
axes[0].set_ylabel('Solid mass fraction')
axes[1].set_ylabel('Solid mass fraction')

T_ax = axes[0].twiny()

xmin,xmax = axes[0].get_xlim()
T_ax.plot(T[:,0],np.repeat(np.nan,N_points),color='black')
T_ax.set_xlim(track.T(xmin,0),track.T(xmax,0))
T_ax.set_xscale('log')

T_ax2 = axes[1].twiny()
T_ax2.plot(T[:,0],np.repeat(np.nan,N_points),color='black')
T_ax2.set_xlim(track.T(xmin,0),track.T(xmax,0))
T_ax2.set_xscale('log')

T_ax.set_xlabel('Temperature [K]')
T_ax2.set_xticklabels([])
plt.tight_layout()
plt.savefig("Figures/{}solids_fraction.{}".format(folder,filetype))
plt.close()

In [ ]:
#%% Manual Category plot 3-in-1

dh = DataHandler('data/mock_joanna_freeiron/output.h5')
starnames = dh.star_names

fig,axes = plt.subplots(3,1,sharex = True,sharey = True,figsize=(fig_width,fig_width*2.5))
s = 10
for i_star,starname in enumerate(starnames):
    
    star_idx = np.where(dh.star_names == starname)[0][0]
    
    water_mass = dh.output["H2-O"][()][star_idx]
    hydrogen_mass = dh.output["H2"][()][star_idx]
    helium_mass = dh.output["He"][()][star_idx]
    abun=dh.input_abun[star_idx]
    abun_names=dh.abun_names

    theta = dh.theta[star_idx]
    
    Z = np.around(theta[np.where(dh.theta_names=="Z")[0][0]],3)
    
    Fe = abun[np.where(abun_names=="Fe/H")[0][0]]

    Fe = np.around(func.get_abun_solar(Fe, solar_abun["Fe/H"].loc[0]),1)
    
    if i_star == 1:
        
        Fe = 0.0
    
    tot_mass = dh.output["final_mass"][()][star_idx]
    sma = dh.output["final_sma"][()][star_idx]
    gas_mass = dh.output["final_gas_mass"][()][star_idx]
    
    gaseous = gas_mass > gas_lim*tot_mass
    water_rich = (water_mass/tot_mass >= wfrac_lim) & ~gaseous
    rocky = (water_mass/tot_mass < wfrac_lim) & ~gaseous
    
    axes[i_star].scatter(sma[gaseous],tot_mass[gaseous],c="red",s=s,label="Gaseous")
    axes[i_star].scatter(sma[water_rich],tot_mass[water_rich],c="blue",label="Water-rich",s=s)
    axes[i_star].scatter(sma[rocky],tot_mass[rocky],c="black",label="Rocky",s=s)

    axes[i_star].set_xlim(0.09,70)
    axes[i_star].set_ylim(0.01,5000)
    axes[i_star].set_xscale("log")
    axes[i_star].set_yscale("log")
    axes[i_star].set_title("Z={}, [Fe/H]={}".format(Z,Fe))
    
axes[-1].set_xlabel("Semi major Axis [AU]")
axes[1].set_ylabel("Mass [$M_\oplus$]")
axes[0].legend()
fig.tight_layout()
dh.close()

plt.savefig('Figures/{}MR_category_all.{}'.format(folder,filetype))
plt.close()

#%% Plot luminosities

iso = pd.read_csv("mass_lum_isochrones.csv")
mass = iso["mass"].values
logt = iso["logt"].values
lum = iso["lum"].values
colors = ['red','blue','yellow','black','magenta']
filt = mass >= 0.6

mass_plot = np.unique(mass[filt])
plt.figure(figsize = (fig_width,fig_width))

for i,mass_val in enumerate(mass_plot[::2]):
    
    track.M_star = np.array([mass_val*MS2ME])
    track.M_gas = 0.1*(mass_val)**1.4*MS2ME
    t_gas = track.get_disc_lifetime()
    
    plot_filt = mass == mass_val
    
    near_idx = func.find_nearest(10**logt[plot_filt], t_gas)
    plot_tdisc = 10**logt[plot_filt][near_idx]
    
    plt.plot(10**logt[plot_filt],10**lum[plot_filt],
             label="$M_*$ = {} $M_\odot$".format(mass_val),color=colors[i],zorder=0)
    plt.scatter(plot_tdisc,10**lum[plot_filt][near_idx],marker='*',color='black',zorder=1,s=100)
    
xlims = plt.gca().get_xlim()
plt.plot([],[],marker='*',color='black',label = 'Disc lifetime',linestyle='')
plt.xlim(np.min(10**logt),10**7)

plt.xscale('log')
plt.legend()
plt.xlabel("Age [yr]")
plt.ylabel("Luminosity [$L_\odot$]")
plt.savefig("Figures/{}luminosity_time.{}".format(folder,filetype))
plt.close()

In [ ]:
#%% Plot occurrences over mass/feh grid

output_dir = "mock_feh_mass_many_freeiron"
cmap ='plasma'

dh = DataHandler("data/"+output_dir+"/output.h5")
solar_abun = pd.read_csv("solar_abun.csv")

N_star = dh.N_star
N_grid = int(np.sqrt(N_star))

x_idx = np.where(dh.abun_names == "Fe/H")[0][0]
x_abun = dh.input_abun[:,x_idx]
x_abun = func.get_abun_solar(x_abun, solar_abun["Fe/H"].loc[0])

star_mass = dh.theta[:,3]/MS2ME

tot_mass = dh.output['final_mass'][()]
wfrac = dh.output['H2-O'][()]/tot_mass
#wfrac_lim = dh.wfrac_lim
grown = tot_mass > min_mass

gaseous = (dh.output['final_gas_mass'][()] > gas_lim*tot_mass) & grown
rocky = ~gaseous & (wfrac < wfrac_lim) & grown
water_rich = ~gaseous & ~rocky & grown

giant = tot_mass > giant_lim
SE = (tot_mass < giant_lim) & (tot_mass > SE_lowlim)
E = ~giant & ~SE & grown

N_bodies = np.sum(grown,axis = 1)

gas_frac = np.sum(gaseous,axis=1)/N_bodies
gas_frac[gas_frac==0] = np.nan

rocky_frac = np.sum(rocky,axis = 1)/N_bodies
rocky_frac[rocky_frac==0] = np.nan

giant_frac = np.sum(giant,axis = 1)/N_bodies
giant_frac[giant_frac==0] = np.nan

water_rich_frac = np.sum(water_rich,axis = 1)/N_bodies
water_rich_frac[water_rich_frac==0] = np.nan

SE_frac = np.sum(SE,axis = 1)/N_bodies
SE_frac[SE_frac==0] = np.nan

E_frac = np.sum(E,axis = 1)/N_bodies
E_frac[E_frac==0] = np.nan

dh.close()

min_gas = gas_frac[gas_frac!=0].min()
max_gas = gas_frac[gas_frac!=0].max()

min_rocky = rocky_frac[rocky_frac!=0].min()
max_rocky = rocky_frac[rocky_frac!=0].max()

min_waterrich = water_rich_frac[water_rich_frac!=0].min()
max_waterrich = water_rich_frac[water_rich_frac!=0].max()

min_giant = giant_frac[giant_frac!=0].min()
max_giant = giant_frac[giant_frac!=0].max()

min_SE = SE_frac[SE_frac!=0].min()
max_SE = SE_frac[SE_frac!=0].max()

min_E = E_frac[E_frac!=0].min()
max_E = E_frac[E_frac!=0].max()

vmin = np.min((min_gas,min_waterrich,min_giant,min_SE))
vmax = np.max((max_gas,max_waterrich,max_giant,max_SE))

y_abun = star_mass.reshape(N_grid, N_grid)
x_abun = x_abun.reshape(N_grid, N_grid)

fig,axes = plt.subplots(2,3,figsize = (fig_width*2.3,fig_width+2))

gas_im = axes[0,0].pcolormesh(x_abun, y_abun, gas_frac.reshape(
    N_grid, N_grid), cmap=cmap,vmin=0,vmax=0.1)#norm=mplcolors.LogNorm(vmin=vmin,vmax=vmax))

giant_im=axes[1,0].pcolormesh(x_abun, y_abun, giant_frac.reshape(
    N_grid, N_grid), cmap=cmap,vmin=0,vmax=0.1)#norm=mplcolors.LogNorm(vmin=vmin,vmax=vmax))

rocky_im=axes[0,2].pcolormesh(x_abun, y_abun, rocky_frac.reshape(
    N_grid, N_grid), cmap=cmap,vmin=0,vmax=0.7)#norm=mplcolors.LogNorm(vmin=0.5,vmax=1))

water_rich_im=axes[0,1].pcolormesh(x_abun, y_abun, water_rich_frac.reshape(
    N_grid, N_grid), cmap=cmap,vmin=0,vmax=0.7)#norm=mplcolors.LogNorm(vmin=vmin,vmax=vmax))

SE_im = axes[1,1].pcolormesh(x_abun, y_abun, SE_frac.reshape(
    N_grid, N_grid), cmap=cmap,vmin=0,vmax=0.2)#norm=mplcolors.LogNorm(vmin=vmin,vmax=vmax))

E_im = axes[1,2].pcolormesh(x_abun, y_abun, E_frac.reshape(
    N_grid, N_grid), cmap=cmap,vmin=0,vmax=1)#norm=mplcolors.LogNorm(vmin=0.5,vmax=1))

axes[0,0].set_title('Gaseous')
axes[1,0].set_title('Giant')
axes[0,2].set_title('Rocky')
axes[0,1].set_title('Water-rich')
axes[1,1].set_title('Super-Earth')
axes[1,2].set_title('Earth-Mass')

axes[0,0].set_xlabel('[Fe/H]')
axes[0,1].set_xlabel('[Fe/H]')
axes[0,2].set_xlabel('[Fe/H]')
axes[1,0].set_xlabel('[Fe/H]')
axes[1,1].set_xlabel('[Fe/H]')
axes[1,2].set_xlabel('[Fe/H]')


cbar_gas=fig.colorbar(gas_im, ax=axes[0,0])
cbar_gas.set_label(r"$N/N_{\rm tot}$")

cbar_giant=fig.colorbar(giant_im, ax=axes[1,0])
cbar_giant.set_label(r"$N/N_{\rm tot}$")

cbar_rocky=fig.colorbar(rocky_im, ax=axes[0,2])
cbar_rocky.set_label(r"$N/N_{\rm tot}$")

cbar_water_rich=fig.colorbar(water_rich_im, ax=axes[0,1])
cbar_water_rich.set_label(r"$N/N_{\rm tot}$")

cbar_SE = fig.colorbar(SE_im, ax=axes[1,1])
cbar_SE.set_label(r"$N/N_{\rm tot}$")

cbar_E = fig.colorbar(E_im, ax=axes[1,2])
cbar_E.set_label(r"$N_{\rm E}/N_{\rm tot}$")

fig.add_subplot(111,frameon=False)
plt.tick_params(labelcolor='none', which='both',
                top=False,bottom = False,left=False,right=False)

plt.ylabel("Stellar Mass [$M_\odot$]")
plt.tight_layout()
plt.savefig('Figures/{}composition_frac_grid_ext.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% Plot Mean planet masses as a function of Fe/H

#name_add = ['','_delta3','_delta5','_St1','_St3']
name_add = ['','_delta3','_delta5','_vf1','_vf5']

#labels = ['St = 0.01,$\delta$ = 1e-4',
#          'St = 0.01,$\delta$ = 1e-3',
#          'St = 0.01,$\delta$ = 1e-5',
          #'St = 0.01,$\delta$ = 1e-4,\nNo iron oxidation',
#          'St = 0.1,$\delta$ = 1e-4',
#          'St = 0.001,$\delta$ = 1e-4']

labels = ['$v_f$ = 2 m/s, $\delta$ = 1e-4',
          '$v_f$ = 2 m/s, $\delta$ = 1e-3',
          '$v_f$ = 2 m/s, $\delta$ = 1e-5',
          #'St = 0.01, $\delta$ = 1e-4,\nNo iron oxidation',
          '$v_f$ = 1 m/s, $\delta$ = 1e-4',
          '$v_f$ = 5 m/s, $\delta$ = 1e-4']

colors = ['red','blue','purple','black','brown']

plt.figure(figsize = (fig_width+0.5,fig_width))

feh_lims = np.linspace(-0.5,0.5,21)

N_points = 5

shift = np.linspace(-0.003*N_points,0.003*N_points,N_points)

for i_run,add in enumerate(name_add):

    dh = DataHandler('data/mock_joanna_many{}/output.h5'.format(add))
    
    pl_mass = dh.output['final_mass'][()]
    feh = dh.input_abun[:,np.where(dh.abun_names == 'Fe/H')[0][0]]
    feh = func.get_abun_solar(feh,solar_abun['Fe/H'].loc[0])
    
    giant = (pl_mass > giant_lim)
    #grown = pl_mass > 0.05
    
    #rocky_mass = [np.mean(pl_mass[i][~giant[i] & grown[i]]) for i in range(dh.N_star)]
    #giant_mass = [np.mean(pl_mass[i][giant[i]]) for i in range(dh.N_star)]

    rocky_mass = np.array([np.sum((pl_mass[i][~giant[i]])**2)/np.sum(pl_mass[i][~giant[i]]) for i in range(dh.N_star)])
    giant_mass = np.array([np.sum((pl_mass[i][giant[i]])**2)/np.sum(pl_mass[i][giant[i]]) for i in range(dh.N_star)])
    
    dh.close()
    
    mean_data = np.zeros((5,len(feh_lims)-1))
    
    for i_lim,lims in enumerate(zip(feh_lims[:-1],feh_lims[1:])):
        
        lowlim,uplim=lims
        
        filt = (feh<=uplim) & (feh>=lowlim)
        
        mean_data[0,i_lim] = np.mean([lowlim,uplim])+shift[i_run]
        mean_data[1,i_lim] = np.mean(giant_mass[filt])
        mean_data[2,i_lim] = np.mean(rocky_mass[filt])
        mean_data[3,i_lim] = np.std(giant_mass[filt],ddof=1)
        mean_data[4,i_lim] = np.std(rocky_mass[filt],ddof=1)  
        
    plt.scatter(mean_data[0],mean_data[1],label = labels[i_run],
                color=colors[i_run],s=30)
    
    #plt.errorbar(mean_data[0],mean_data[1],yerr=mean_data[3],
    #             capsize = 3,linestyle='',marker='',color=colors[i_run])
    
    plt.scatter(mean_data[0],mean_data[2],marker='^',
                color=colors[i_run],s=30)
    
    #plt.errorbar(mean_data[0],mean_data[2],yerr=mean_data[4],
    #             capsize = 3,linestyle='',marker='',color=colors[i_run])

ymin,ymax=plt.gca().get_ylim()

plt.plot([],[],marker='^',color='grey',linestyle='',label='Non-giant planet')
plt.plot([],[],marker='o',color='grey',linestyle='',label='Giant planet')
#plt.vlines(feh_lims,ymin,ymax,color='black',alpha=0.2)
plt.xlabel('[Fe/H]')
plt.ylabel(r'Planet mass [$M_\oplus$]')
plt.yscale('log')
plt.legend(loc='center left',bbox_to_anchor=(0.55, 0.56),ncol=1,fontsize=11)
plt.tight_layout()
plt.savefig('Figures/{}Mpl_vs_feh.{}'.format(folder,filetype))
plt.close()

#%% Plot Mg/Fe and Mg/H over Fe/H for all three populations

def line(x,k,m):
    
    return k*x+m

data = pd.read_csv('SAPP_total_data_Jesper_final_snr_cut_50.csv')
solar_abun = pd.read_csv('solar_abun.csv')
fit_param = np.loadtxt('fit_param.txt')
full_param = np.array([0.76447892, 0.03344476])
min_feh = np.min(data['feh_reso_bay'])
max_feh = np.max(data['feh_reso_bay'])
feh_plot = np.linspace(min_feh,max_feh,100)
cats = ['Thin_disc','Thick_disc','Halo']
cat_labels = [r'$\alpha$-poor', r'$\alpha$-rich','Halo']
colors = ['blue','red','black']

solar_mgh = solar_abun['Mg/H'].loc[0]
solar_feh = solar_abun['Fe/H'].loc[0]

fig,axes = plt.subplots(2,1,figsize = (fig_width,fig_width*1.2),sharex = True)

for i_cat,cat_name in enumerate(cats):
    
    filt = data['Cat'] == cat_name
    
    feh = data['feh_reso_bay'].loc[filt]
    mgfe = data['mgfe_reso_bay'].loc[filt]
    
    mgh = func.xfe_to_xh(mgfe, feh, solar_mgh, solar_feh)
    
    mgfe_plot = func.get_ironratio(line(feh_plot,*fit_param[i_cat]),
                                   feh_plot, solar_mgh,log_scale=True)
    
    min_feh = np.min(feh)
    max_feh = np.max(feh)
    feh_filt = (feh_plot <= max_feh) & (feh_plot >= min_feh)
    cat_plot = feh_plot
    
    axes[0].scatter(feh,mgfe,color=colors[i_cat],s=5)
    axes[0].plot(cat_plot,mgfe_plot,color = colors[i_cat])
    axes[1].scatter(feh,mgh,color=colors[i_cat],s=5,label = cat_labels[i_cat])
    axes[1].plot(cat_plot,line(cat_plot,*fit_param[i_cat]),color=colors[i_cat])

mgfe_fit_full = func.get_ironratio(line(feh_plot,*full_param),
                               feh_plot, solar_mgh,log_scale=True)

axes[0].plot(feh_plot,mgfe_fit_full,color='black',linestyle='--')
axes[1].plot(feh_plot,line(feh_plot,*full_param),color='black',linestyle='--')
axes[1].plot(feh_plot,feh_plot,color = 'grey',linestyle='-.',label='[Mg/H]=[Fe/H]')
axes[0].text(0.95,0.9, '(a)',horizontalalignment='right',transform=axes[0].transAxes)
axes[1].text(0.95,0.7, '(b)',horizontalalignment='right',transform=axes[1].transAxes)
axes[0].set_ylabel('[Mg/Fe]')
axes[1].set_xlabel('[Fe/H]')
axes[1].set_ylabel('[Mg/H]')
axes[1].legend(loc=2,fontsize=12)

text_labels = [r'\alpha-poor',r'\alpha-rich','Halo','Full']

for i_text,text in enumerate(text_labels):
    
    if i_text == 3:
        
        k = full_param[0]
        m = full_param[1]
    
    else:
        
        k = fit_param[i_text,0]
        m = fit_param[i_text,1]
    
    axes[1].text(0.35,0.2-0.06*i_text,
                 r'$k_\mathrm{{{name}}}={val:.2f}$,'.format(name=text,val=k),
                 transform=axes[1].transAxes,fontsize=14,
                 verticalalignment='bottom',horizontalalignment='left')
    
    axes[1].text(0.67,0.2-0.06*i_text,
                 r'$m_\mathrm{{{name}}}={val:.2f}$'.format(name=text,val=m),
                 transform=axes[1].transAxes,fontsize=14,
                 verticalalignment='bottom',horizontalalignment='left')    

plt.tight_layout()
plt.savefig('Figures/{}mgfe_mgh_feh.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% Plot occurrences with Mg/Fe and Fe/H

cat_labels = [r'$\alpha$-poor',r'$\alpha$-rich',"Halo"]
cat_file = ["thin","thick","halo"]
markers = ["o","^","s"]
type_names = ['Gaseous', 'Water-rich', 'Rocky', 'Giant','Super-Earth', 'Earth-mass']
colors = ['red','blue','black','teal','turquoise','purple']
s=10

fig,axes = plt.subplots(2,3,figsize = (fig_width*2.3,fig_width+2),sharex=False,sharey=True)

min_vals = np.zeros((3,6))
max_vals = np.zeros((3,6))
ims = []
y_name = 'Mg/Fe'

for i_cat,name in enumerate(cat_file):

    output_dir = "data/{}_freeiron_full/output.h5".format(name)
    
    dh = DataHandler(output_dir)
    solar_abun = pd.read_csv("solar_abun.csv")
    
    N_star = dh.N_star
    N_grid = int(np.sqrt(N_star))
    
    input_feh = dh.input_abun[:,np.where(dh.abun_names == "Fe/H")[0][0]]
    
    x_abun = func.get_abun_solar(input_feh, solar_abun["Fe/H"].loc[0]) 
    
    y_name_H = y_name.split("/")[0]+"/H"
    y_abun = dh.input_abun[:,np.where(dh.abun_names == y_name_H)[0][0]]
    y_abun = func.get_ironratio(y_abun,input_feh,solar_abun[y_name_H].loc[0])
    
    tot_mass = dh.output['final_mass'][()]
    gas_mass = dh.output['final_gas_mass'][()]
    wfrac = dh.output['H2-O'][()]/tot_mass
    #wfrac_lim = dh.wfrac_lim
    grown = tot_mass > min_mass
    
    gaseous = (gas_mass > gas_lim*tot_mass) & grown
    rocky = ~gaseous & (wfrac < wfrac_lim) & grown
    water_rich = ~gaseous & ~rocky & grown
    
    giant = tot_mass > giant_lim
    SE = (tot_mass < giant_lim) & (tot_mass > SE_lowlim)
    E = ~giant & ~SE & grown
    
    N_bodies = np.sum(grown,axis = 1)
    
    gas_frac = np.sum(gaseous,axis=1)/N_bodies
    gas_frac[np.isnan(gas_frac)] = 0
    
    giant_frac = np.sum(giant,axis = 1)/N_bodies
    giant_frac[np.isnan(giant_frac)] = 0
    
    rocky_frac = np.sum(rocky,axis = 1)/N_bodies
    rocky_frac[np.isnan(rocky_frac)] = 0
    
    water_rich_frac = np.sum(water_rich,axis = 1)/N_bodies
    water_rich_frac[np.isnan(water_rich_frac)] = 0
    
    SE_frac = np.sum(SE,axis = 1)/N_bodies
    SE_frac[np.isnan(SE_frac)] = 0
    
    E_frac = np.sum(E,axis = 1)/N_bodies
    E_frac[np.isnan(E_frac)] = 0
    
    dh.close()
    
    count = (gas_frac,water_rich_frac,rocky_frac,giant_frac,SE_frac,E_frac)
    
    #colors = iter(plt.cm.tab20c(np.linspace(0, 1,len(count))))
    
    for i_type,frac in enumerate(count):
        
        #c=next(colors)
        #filt = (x_abun <0.01) & (x_abun>-0.01)

        axes[0,i_cat].scatter(x_abun,frac,label=type_names[i_type],s=10,color=colors[i_type])
        axes[1,i_cat].scatter(y_abun,frac,label=type_names[i_type],s=10,color=colors[i_type])
    
    axes[0,i_cat].set_title(cat_labels[i_cat])
    axes[0,i_cat].set_yscale('log')
    axes[0,i_cat].set_xlim(-1.6,0.5)
    axes[1,i_cat].set_xlim(-0.15,0.5)
    axes[0,i_cat].set_xlabel('[Fe/H]')
    axes[1,i_cat].set_xlabel('[Mg/Fe]')
    
axes[0,0].legend(fontsize=14,loc='upper left',bbox_to_anchor=(-0.05,1),frameon=False,handletextpad=0.01)

fig.add_subplot(111,frameon=False)
plt.tick_params(labelcolor='none', which='both', 
                top=False, bottom=False, left=False, right=False)

plt.ylabel('Fraction of planets',labelpad=20)
plt.tight_layout()
plt.savefig('Figures/{}occ_mgfe_feh.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% Plot 2D histogram of planet mass and feh

colors = ['blue','red','black']
cat_file = ['thin','thick','halo']

cbars = []
pl_mass_data = []
feh_data = []

plt.figure(figsize=(fig_width,fig_width-1))

for i_cat,name in enumerate(cat_file):

    output_dir = "data/{}_full/output.h5".format(name)
    
    dh = DataHandler(output_dir)
    solar_abun = pd.read_csv("solar_abun.csv")
    N_star = dh.N_star
    n_feh = dh.input_abun[:,np.where(dh.abun_names == 'Fe/H')[0][0]]
    
    feh = func.get_abun_solar(n_feh, solar_abun['Fe/H'].loc[0])
    
    pl_mass = dh.output['final_mass'][()].flatten()

    dh.close()
    
    pl_mass_data = np.append(pl_mass_data,pl_mass)
    feh_data = np.append(feh_data,np.repeat(feh,dh.N_bodies))

plt.hist2d(pl_mass_data,feh_data,
                   bins=[np.logspace(-2,3.5,40),np.linspace(-2,0.6,40)],cmap='plasma',
                   norm=mplcolors.LogNorm())

plt.yticks(ticks=np.array([-1.5,-1,-0.5,0,0.5]))
plt.xscale('log')
plt.ylabel('[Fe/H]')
plt.xlabel(r'Planet Mass [$M_\oplus$]')
cbar = plt.colorbar()
cbar.set_label('Number of planets')
plt.tight_layout()
plt.savefig('Figures/{}feh_mass_2dhist.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% Plot bar plot of occurrences

cat_labels = [r'$\alpha$-poor',r'$\alpha$-rich',"Halo"]
cat_file = ["thin","thick","halo"]
colors = ['blue','red','black']
width = 0.5
pos = [-width,0,width]
pos2 = [-0.25,0.25,0.75]
x_pos = np.linspace(0,10,6)

plt.figure(figsize=(fig_width,fig_width))

full_data = np.zeros((3,6))

for i_cat,name in enumerate(cat_file):

    output_dir = "data/{}_freeiron_full/output.h5".format(name)
    
    dh = DataHandler(output_dir)
    solar_abun = pd.read_csv("solar_abun.csv")
    
    N_star = dh.N_star
    
    M_star = dh.theta[:,np.where(dh.theta_names == 'M_star')[0][0]]/MS2ME
    tot_mass = dh.output['final_mass'][()]
    gas_mass = dh.output['final_gas_mass'][()]
    wfrac = dh.output['H2-O'][()]/tot_mass
    #wfrac_lim = dh.wfrac_lim
    #M_star_filt = M_star < M_star_lim
    #M_star_filt = np.repeat(M_star_filt[:,np.newaxis],dh.N_bodies,axis=1)
    
    grown = (tot_mass > min_mass)

    gaseous = grown & (gas_mass > gas_lim*tot_mass)
    rocky = ~gaseous & (wfrac < wfrac_lim) & grown 
    icy = ~gaseous & ~rocky & grown
    
    giant = (tot_mass > giant_lim) & grown
    SE = (tot_mass < SE_uplim) & (tot_mass > SE_lowlim) 
    E = ~giant & ~SE & grown 
    
    N_bodies = np.sum(grown,axis = 1)
    
    gas_frac = np.sum(gaseous)
    
    rocky_frac = np.sum(rocky)
    
    icy_frac = np.sum(icy)
    
    giant_frac = np.sum(giant)
    
    SE_frac = np.sum(SE)
    
    E_frac = np.sum(E)
    
    dh.close()
    
    count = (gas_frac,rocky_frac,icy_frac,giant_frac,SE_frac,E_frac)
    
    plt.bar(x_pos+pos[i_cat],count/np.sum(N_bodies),width=width/2-0.01,
            color = colors[i_cat],label=cat_labels[i_cat],edgecolor='black',align='edge')
    
    output_dir_nomassive = "data/{}_freeiron_nomassive/output.h5".format(name)
    dh = DataHandler(output_dir_nomassive)
    solar_abun = pd.read_csv("solar_abun.csv")
    
    N_star = dh.N_star
    
    M_star = dh.theta[:,np.where(dh.theta_names == 'M_star')[0][0]]/MS2ME
    tot_mass = dh.output['final_mass'][()]
    gas_mass = dh.output['final_gas_mass'][()]
    wfrac = dh.output['H2-O'][()]/tot_mass
    #wfrac_lim = dh.wfrac_lim
    #M_star_filt = M_star < M_star_lim
    #M_star_filt = np.repeat(M_star_filt[:,np.newaxis],dh.N_bodies,axis=1)
    
    grown = (tot_mass > min_mass)

    gaseous = grown & (gas_mass > gas_lim*tot_mass)
    rocky = ~gaseous & (wfrac < wfrac_lim) & grown 
    icy = ~gaseous & ~rocky & grown
    
    giant = (tot_mass > giant_lim) & grown
    SE = (tot_mass < SE_uplim) & (tot_mass > SE_lowlim) 
    E = ~giant & ~SE & grown 
    
    N_bodies = np.sum(grown,axis = 1)
    
    gas_frac = np.sum(gaseous)
    
    rocky_frac = np.sum(rocky)
    
    icy_frac = np.sum(icy)
    
    giant_frac = np.sum(giant)
    
    SE_frac = np.sum(SE)
    
    E_frac = np.sum(E)
    
    dh.close()
    
    count = (gas_frac,rocky_frac,icy_frac,giant_frac,SE_frac,E_frac)

    plt.bar(x_pos+pos2[i_cat],count/np.sum(N_bodies),width=width/2-0.01,
            color = 'none',hatch='//',edgecolor=colors[i_cat],align='edge')

plt.bar(0,0,color='none',hatch='//',label='$M_*\leq1.4M_\odot$',edgecolor='grey')
plt.xticks(x_pos,labels = ['Gaseous','Rocky','Water-rich',
                                  'Giant','Super Earth', 'Earth-mass'],rotation =45)
plt.yscale('log')
plt.ylabel(r'Fraction of planets')
plt.legend(bbox_to_anchor=(0.05,1.15,1,0),ncol=4,fontsize=9)
plt.tight_layout()
plt.savefig('Figures/{}occurrence_bar.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% 2D histogram of cmf

earth_cmf = 0.325
mars_cmf = 0.25
cmf_data = []
feh_data = []
#plt.figure(figsize=(fig_width+4,fig_width+2))
fig,axes=plt.subplots(2,3,sharex=True,sharey=True,figsize=(fig_width*2.3,fig_width+2))
labels = [r'$\alpha$-poor',r'$\alpha$-rich','Halo']

for i_cat,cat in enumerate(['thin','thick','halo']):
    
    for i_model,model in enumerate(['freeiron_','']):
        
        dh = DataHandler('data/{}_{}full/output.h5'.format(cat,model))
        cmf = dh.get_cmf()
        mass = dh.output['final_mass'][()]
        solid_mass = dh.output['final_solid_mass'][()]
        r_init = dh.input['r0_init'][()]
        feh = dh.input_abun[:,np.where(dh.abun_names == 'Fe/H')[0][0]]
        mgh = dh.input_abun[:,np.where(dh.abun_names == 'Mg/H')[0][0]]
        mgfe = func.get_ironratio(mgh, feh, solar_abun['Mg/H'].loc[0])
        feh = func.get_abun_solar(feh, solar_abun['Fe/H'].loc[0])
        
        filt = (mass>min_mass) & (mass <giant_lim)
        dh.close()
    
        #If looking at Mg/Fe, change min bin to -0.1
        #FeH: -1.5
        im=axes[i_model,i_cat].hist2d(np.repeat(feh,dh.N_bodies)[filt.flatten()],cmf[filt],
                           bins=[np.linspace(-1.5,0.6,35),np.linspace(0.15,0.4,35)],
                           cmap='plasma',norm=mplcolors.LogNorm())
        axes[i_model,i_cat].text(0.3,0.85, labels[i_cat],
                         horizontalalignment='right',
                         transform=axes[i_model,i_cat].transAxes,fontsize=16)
    
        axes[i_model,i_cat].scatter(0,earth_cmf,color='black',s=30,label='Earth',edgecolor='grey')
        axes[i_model,i_cat].scatter(0,mars_cmf,color='black',s=30,label='Mars',marker='^',edgecolor='grey')
        
        fig.colorbar(im[3],ax=axes[i_model,i_cat])
        
        axes[i_model,i_cat].set_xlabel('[Fe/H]',fontsize=16)
    
axes[0,0].legend(fontsize=12,loc='lower left')
axes[1,1].set_title('Including Iron oxidation')
axes[0,1].set_title('Excluding Iron oxidation')

axes[0,0].set_ylabel('Core mass fraction')
axes[1,0].set_ylabel('Core mass fraction')
plt.tight_layout()
plt.savefig('Figures/{}cmf_2dhist.{}'.format(folder,filetype),dpi=400)
plt.close()

In [ ]:
#%% Large disc plot

dh = DataHandler('data/sun_joanna_largedisc/output.h5')

water_mass = dh.output["H2-O"][()][0]
abun=dh.input_abun[0]
abun_names=dh.abun_names

theta = dh.theta


Z = np.around(theta[np.where(dh.theta_names=="Z")[0][0]],3)

#Si = abun[np.where(abun_names=="Si/H")[0][0]]
#Mg = abun[np.where(abun_names=="Mg/H")[0][0]]
Fe = abun[np.where(abun_names=="Fe/H")[0][0]]

#Si = np.around(func.get_abun_solar(Si, self.solar_abun["Si/H"].loc[0]),1)
#Mg = np.around(func.get_abun_solar(Mg, self.solar_abun["Mg/H"].loc[0]),1)
Fe = np.around(func.get_abun_solar(Fe, solar_abun["Fe/H"].loc[0]),1)

if Fe==0:
    
    Fe = 0.0
    
tot_mass = dh.output["final_mass"][()][0]
sma = dh.output["final_sma"][()][0]
gas_mass = dh.output['final_gas_mass'][()][0]

dh.close()

gaseous=(gas_mass/tot_mass >=gas_lim)
icy = (water_mass/tot_mass >= wfrac_lim) & ~gaseous
rocky = (water_mass/tot_mass < wfrac_lim) & ~gaseous

plt.figure(figsize=(fig_width,fig_width))
plt.scatter(sma[gaseous],tot_mass[gaseous],c="red",s=5,label="Gaseous")
plt.scatter(sma[icy],tot_mass[icy],c="blue",label="Icy",s=5)
plt.scatter(sma[rocky],tot_mass[rocky],c="black",label="Rocky",s=5)

plt.legend()
plt.xlim(0.09,60)
plt.ylim(0.1,5000)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Semi major Axis [AU]")
plt.ylabel("Mass [$M_\oplus$]")
plt.title("Z={}, [Fe/H]={}".format(Z,Fe))
plt.tight_layout()
plt.savefig('Figures/{}MR_category_largedisc.{}'.format(folder,filetype))
plt.close()

In [ ]:
#%% Mass v Feh of stars

data = pd.read_csv('SAPP_total_data_Jesper_final_snr_cut_50.csv')

cats = ['Thin_disc','Thick_disc','Halo']
cat_labels = [r'$\alpha$-poor', r'$\alpha$-rich','Halo']
colors = ['blue','red','black']
   
plt.figure(figsize=(fig_width,fig_width))

for i_cat,cat_name in enumerate(cats):
    
    filt = data['Cat'] == cat_name
    
    feh = data['feh_reso_bay'].loc[filt]
    feh_err = data['feh_reso_bay'].loc[filt]
    mass = data['mass_wa_bay'].loc[filt]
    mass_err = data['mass_err_wa_bay'].loc[filt]

    plt.errorbar(feh,mass,yerr=mass_err,linestyle='',marker='o',
                 color=colors[i_cat],label=cat_labels[i_cat],markersize=5)

plt.legend()
plt.xlabel('[Fe/H]')
plt.ylabel('Stellar mass [$M_\odot$]')
plt.savefig('Figures/{}feh_stellar_mass.{}'.format(folder,filetype))
plt.close()